# VacationPy

In [1]:
####################################################
#00     I/O                                        #
#   a- import libraries and api key.               #
#   b- read input into df and configure api key.   #
####################################################

#a
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from api_keys import g_key

#b
city_weather_file = "output_data/cities.csv"
city_weather = pd.read_csv(city_weather_file)

gmaps.configure(api_key=g_key)

In [2]:
#############################################################
#01     Humidity Map                                        #
#   a- set locations and humidity as "heat" for map.        #
#   b- figure base map, set heat layer, and add it.         #
#   c- display map and save.                                #
#############################################################

#a
locations = city_weather[["lat", "lng"]].astype(float)
humidity = city_weather["humid"]#.astype(float)

#b
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.7)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=100, point_radius = 2)
fig.add_layer(heat_layer)

#c
fig

Figure(layout=FigureLayout(height='420px'))

In [3]:
################################################################
#02      Vacation Weather                                      #
#   a- create new df, filtering cities to my weather           #
#      specifications for vacation spots.                      #
#   b- drop any rows with missing data from new vacation df.   #
################################################################

#a
vacation_weather = city_weather[(city_weather["temp"] > 80) & 
                                (city_weather["clouds"] >= 50) & (city_weather["humid"] >= 85)]
#b
vacation_weather = vacation_weather.dropna()
vacation_weather.reset_index(inplace=True,drop=True)
vacation_weather

,city,temp,humid,wind,clouds,lat,lng,maxtemp,country,local_date_time
0,mahibadhoo,81.41,86,21.41,100,3.7833,72.9667,81.41,MV,2021-05-11 17:08:41
1,qui nhon,81.34,87,8.57,62,13.7667,109.2333,81.34,VN,2021-05-11 19:09:20
2,kulhudhuffushi,81.01,87,9.75,100,6.6221,73.0700,81.01,MV,2021-05-11 17:09:43
3,limbang,83.21,89,2.30,75,4.7500,115.0000,84.00,MY,2021-05-11 20:09:18
4,muli,80.53,87,16.44,100,2.9167,73.5667,80.53,MV,2021-05-11 17:11:33
5,gigmoto,80.49,86,6.55,99,13.7796,124.3900,80.49,PH,2021-05-11 20:12:00
6,dhidhdhoo,81.27,86,12.10,100,6.8833,73.1000,81.27,MV,2021-05-11 17:12:06
7,male,81.79,86,22.97,100,4.1748,73.5089,81.79,MV,2021-05-11 17:12:45
8,sungairaya,80.20,87,3.78,97,0.7000,108.9000,80.20,ID,2021-05-11 19:13:18
9,dicabisagan,80.67,86,7.38,100,17.0818,122.4157,80.67,PH,2021-05-11 20:13:30


In [4]:
############################################################################
#03    Hotel Df                                                            #
#   a- creaate hotel_df, add name col, set base_url and parameters.        #
#      create list in case missing hotel info to delete row.               #
#   b- for each location in df, call google for nearby hotel, add to df.   #
#   c- if missing hotel info, delete records from hotels_df.               #
############################################################################

#a
hotel_df = vacation_weather
hotel_df["Hotel Name"] = ""
params = {"radius": 5000, "keyword": "hotels", "key": g_key}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
del_indexes = []

#b
for x, row in hotel_df.iterrows():
    lat = row["lat"]                                           # get lat, lng from df
    lng = row["lng"]
    params["location"] = f"{lat},{lng}"                        # set location at each iteration 
    
    response = requests.get(base_url, params=params).json()    #call google for hotel name
    
    try:
        hotel_df.loc[x, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        del_indexes.append(x)
#c
if len(del_indexes) > 0:
    hotel_df.drop(del_indexes, inplace=True)
    hotel_df.reset_index(inplace=True, drop=True)

In [5]:
##############################################################
#04     Hotel InfoBox                                        #
#   - prewritten block of code for step 05 Marker Overlay.   #
#      - setup infobox data in hotel_info.                   #
#      - create lat,lng df for marker locations.             #
##############################################################

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [6]:
###################################################
#05      Marker Overlay                           #
#   a- assign marker layer to variable markers.   #
#   b- add layer to map and display.              #
###################################################

#a
markers = gmaps.marker_layer(locations,info_box_content =hotel_info)
#b
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))